In [1]:
from model import param, params, logit, survey
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

In [2]:
# load data
sago = survey()
sago.load()
attr_labels = ['liv_rpa', 'liv_ri', 'liv_chsld', 'sup_pri', 'sup_npf', 'svc_inf', 'svc_avq', 'svc_avd', 'cons', 'wait']
#attr_labels = ['liv_rpa', 'liv_ri', 'liv_chsld', 'svc_inf', 'svc_avq', 'svc_avd', 'cons']
attributes = sago.extract_attributes(attr=attr_labels)
cs = sago.extract_choices()

                count         mean          std         min          25%  \
choice         9684.0     2.090768     0.832150    1.000000     1.000000   
svc_inf1       9684.0     6.584761     6.336579    0.000000     2.165000   
svc_inf2       9684.0     6.653507     6.329680    0.000000     2.165000   
svc_inf3       9684.0    11.158558     8.906957    0.000000     2.165000   
svc_avq1       9684.0    22.337573    22.594798    1.082500     6.495000   
svc_avq2       9684.0    22.524808    22.834558    1.082500     6.495000   
svc_avq3       9684.0    38.792601    32.205421    1.082500     7.577500   
svc_avd1       9684.0    29.626563    19.776153    2.165000    12.990000   
svc_avd2       9684.0    28.887905    19.785529    2.165000    11.907500   
svc_avd3       9684.0    44.996297    22.638309    2.165000    23.815000   
need_inf       9684.0    12.666055     7.650531    3.247500     8.660000   
need_avq       9684.0    42.774623    28.525789    8.660000    17.590625   
need_avd    

In [3]:
len(sago.data)

9684

In [4]:
len(sago.data)/4

2421.0

In [5]:
df_stat = pd.DataFrame(index=attr_labels,columns=['mean','std','min','max'])
for s in attr_labels:    
    df_stat.loc[s,'mean'] = attributes[s].mean()
    df_stat.loc[s,'std'] = attributes[s].std()
    df_stat.loc[s,'min'] = attributes[s].min()
    df_stat.loc[s,'max'] = attributes[s].max()

In [6]:
df_stat

,mean,std,min,max
liv_rpa,0.41381,0.492515,0.0,1.0
liv_ri,0.0559,0.229728,0.0,1.0
liv_chsld,0.117582,0.322113,0.0,1.0
sup_pri,0.335399,0.47213,0.0,1.0
sup_npf,0.274783,0.446405,0.0,1.0
svc_inf,1.608953,1.063622,0.0,3.480471
svc_avq,2.711765,1.233658,0.079273,4.51009
svc_avd,3.220707,0.913951,0.77242,4.156811
cons,3.084672,1.298418,0.0,5.521461
wait,0.250803,0.291266,0.0,1.333333


In [1]:
# initialize parameters
init_pars = params()
#shifter_labels = ['constant']
#shifter_labels = ['constant','needs_level']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','prob_adl']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','prob_adl','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','med_adl_prob','high_adl_prob']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','med_adl_prob','high_adl_prob','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss','ln_income']
shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss','ln_income','age_norm']

for s in ['liv_rpa','liv_ri','liv_chsld']:
     for h in shifter_labels:
          if h=='ln_income':
               p = param(s,'ln_income',value = 0,free=False)
          else:
               p = param(s,h,value = 0)
          init_pars.add_param(p)

for s in ['sup_pri','sup_npf']:
     for h in shifter_labels:
          if h=='constant':
               p = param(s,'constant',value = -2)
          elif (h=='ln_income') & (s=='sup_pri'):
               p = param(s,'ln_income',value = 0)
          else :
               p = param(s,h,value = 0, free=False)
               #p = param(s,h,value = 0)
          init_pars.add_param(p)


for h in shifter_labels:
     if h=='constant':
          p = param('svc_inf','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_inf','ln_income',value = 0,free=False)
     else :
          p = param('svc_inf',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('svc_avq','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_avq','ln_income',value = 0,free=False)
     else :
          p = param('svc_avq',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('svc_avd','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_avd','ln_income',value = 0,free=False)
     else :
          p = param('svc_avd',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('cons',h,value = -2)
     else :
          p = param('cons',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('wait',h,value = -2)
     else :
          p = param('wait',h,value = 0,free=False)
          #p = param('wait',h,value = 0)
     init_pars.add_param(p)

shifters = sago.extract_shifters(shifter_labels)     
n_attr = len(sago.attr_labels) 

NameError: name 'params' is not defined

In [146]:
model = logit(with_uh = False)
model.set_params(init_pars)
for s in sago.attr_labels:  
    model.set_shifters(s,shifters)
model.set_choices(cs)

#ctr = [0,0,0,0,0,1,1,1,1,-1]
ctr = [0,0,0,0,0,0,0,0,0,0]
#ctr = [0,0,0,1,1,1,1]
ctrs = dict(zip(sago.attr_labels,ctr))
model.set_attributes(attributes, sago.attr_labels, ctrs)
model.set_draws(n_d = 100)

In [147]:
model.estimate(scn_name = 'pre_uh_test')

iter =  0 , loglike =  -7.0533130149626775
current theta:
[-0.05605158 -0.03683364 -0.01454974 -0.00739206  0.02388866  0.01907985
  0.0061275   0.00261451  0.0395634   0.03226779  0.00964166  0.00395635
 -1.98814546  0.04422491 -2.002155   -1.86801782  0.08941754  0.03458253
  0.01605894  0.4998451  -1.85405135  0.09161669  0.03588817  0.01650636
  0.5522502  -1.84242448  0.09919538  0.04031367  0.01900002  0.59759866
 -1.98769121  0.01476739  0.00321834  0.00358595  0.03926512 -1.97039901]
iter =  1 , loglike =  -5.77748710559371
current theta:
[-1.32822531e-01 -7.93974138e-02 -3.44159707e-02 -1.69772000e-02
  4.25426377e-02  3.52835675e-02  1.03639662e-02  4.79068547e-03
  5.26646373e-02  4.70273231e-02  1.23605099e-02  1.56534556e-03
 -1.91393378e+00  3.29190401e-01 -1.93312001e+00 -1.82035415e+00
  1.19780348e-01  4.95804674e-02  1.69134097e-02  6.64411962e-01
 -1.83923232e+00  9.82712471e-02  3.69962791e-02  1.30274655e-02
  5.90461482e-01 -1.81130646e+00  1.21632936e-01  4.99306

In [148]:
init_pars.table()

,group,label,value,stderr,tvalue,free
0,liv_rpa,constant,-0.93857,0.073194,-12.823087,True
1,liv_rpa,needs_level,0.833992,0.119753,6.964262,True
2,liv_rpa,prob_help_adl,-0.21765,0.078845,-2.76048,True
3,liv_rpa,help_adl_miss,-0.163541,0.067866,-2.409778,True
4,liv_rpa,ln_income,0,NaN,NaN,False
5,liv_ri,constant,-0.492528,0.342118,-1.439644,True
6,liv_ri,needs_level,1.122217,0.436828,2.569016,True
7,liv_ri,prob_help_adl,-0.371731,0.195417,-1.902239,True
8,liv_ri,help_adl_miss,-0.196654,0.179189,-1.097465,True
9,liv_ri,ln_income,0,NaN,NaN,False


In [25]:
# initialize parameters
init_pars = params()

shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','prob_adl']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','prob_adl','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','med_adl_prob','high_adl_prob']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','med_adl_prob','high_adl_prob','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss','ln_income','age_norm']

for s in ['liv_rpa','liv_ri','liv_chsld']:
     for h in shifter_labels:
          #if h=='ln_income':
          #     p = param(s,'ln_income',value = 0,free=False)
          #else:
          p = param(s,h,value = 0)
          init_pars.add_param(p)

for s in ['sup_pri','sup_npf']:
     for h in shifter_labels:
          if h=='constant':
               p = param(s,'constant',value = -2)
          #elif (h=='ln_income') & (s=='sup_pri'):
          #     p = param(s,'ln_income',value = 0)
          else :
          #     p = param(s,h,value = 0, free=False)
                p = param(s,h,value = 0)
          init_pars.add_param(p)


for h in shifter_labels:
     if h=='constant':
          p = param('svc_inf','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_inf','ln_income',value = 0,free=False)
     else :
          p = param('svc_inf',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('svc_avq','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_avq','ln_income',value = 0,free=False)
     else :
          p = param('svc_avq',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('svc_avd','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_avd','ln_income',value = 0,free=False)
     else :
          p = param('svc_avd',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('cons',h,value = -2)
     else :
          p = param('cons',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('wait',h,value = -2)
     else :
          #p = param('wait',h,value = 0,free=False)
          p = param('wait',h,value = 0)
     init_pars.add_param(p)
shifters = sago.extract_shifters(shifter_labels)     
n_attr = len(sago.attr_labels) 

In [26]:
init_pars.restart_from('pre_uh_test')
for d in range(n_attr):
     for u in range(d,n_attr):
      if (d==u) & (u==0):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = -0.16441865431692776,free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
      elif (d==u) & (u==1):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = 0.31138671551666103,free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.000001),free=False)
      elif (d==u) & (u==2):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = 0.9727600255515132,free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
      elif (d==u) & (u==3):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.2),free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.00001),free=False)
      elif (d==u) & (u==4):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.2),free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.00001),free=False)
      elif (d==u) & (u==5):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = -9.314522709058613,free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.00001),free=False)
      elif (d==u) & (u==6):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = -12.87177510065236,free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.00001),free=False)
      elif (d==u) & (u==7):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = -0.6179462121322983,free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.000001),free=False)
      elif (d==u) & (u==8):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = 0.5811793297594894,free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.000001),free=False)
      elif (d==u) & (u==9):
            #p = param('uh','ln_sig_'+str(d)+str(u), value = 0.3586444195657937,free=True)
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.000001),free=False)
      #elif (d==0) & (u==1):
            #p = param('uh','rho_'+str(d)+str(u),value = 1.0093251504513487)
            #p = param('uh','rho_'+str(d)+str(u), value = 0,free=True)
      #elif (d==0) & (u==2):
            #p = param('uh','rho_'+str(d)+str(u),value = 0.9405090563378576)
      #      p = param('uh','rho_'+str(d)+str(u), value = 0,free=True)
      #elif (d==1) & (u==2):
            #p = param('uh','rho_'+str(d)+str(u),value = 2.635268514781)
      #      p = param('uh','rho_'+str(d)+str(u), value = 0,free=True)
      #elif (d==5) & (u==6):
            #p = param('uh','rho_'+str(d)+str(u),value = 0)
      #      p = param('uh','rho_'+str(d)+str(u), value = 0,free=False)
      #elif (d==5) & (u==7):
            #p = param('uh','rho_'+str(d)+str(u),value = 0)
      #      p = param('uh','rho_'+str(d)+str(u), value = 0,free=False)
      #elif (d==6) & (u==7):
            #p = param('uh','rho_'+str(d)+str(u),value = 0)
      #      p = param('uh','ln_sig_'+str(d)+str(u), value = 0,free=False)
      else:
            p = param('uh','rho_'+str(d)+str(u),value = 0,free=False)
      init_pars.add_param(p)

#init_pars.restart_from('uh_prob_help_income_corr')
#init_pars.pars[92].value = np.log(0.2)
#init_pars.pars[95].value = np.log(0.2)
#init_pars.pars[99].value = np.log(0.2)
#init_pars.pars[104].value = np.log(0.2)
#init_pars.pars[67].value = np.log(0.000001)
#init_pars.pars[74].value = np.log(0.000001)
#init_pars.pars[80].value = np.log(0.2)
#init_pars.pars[85].value = np.log(0.2)
#init_pars.pars[74].value = np.log(0.2)
#init_pars.pars[77].value = np.log(0.2)
#init_pars.pars[84].value = np.log(0.2)


    Unnamed: 0      group          label     value    stderr     tvalue   free
0            0    liv_rpa       constant -0.938570  0.073194 -12.823087   True
1            1    liv_rpa    needs_level  0.833992  0.119753   6.964262   True
2            2    liv_rpa  prob_help_adl -0.217650  0.078845  -2.760480   True
3            3    liv_rpa  help_adl_miss -0.163541  0.067866  -2.409778   True
4            4    liv_rpa      ln_income  0.000000       NaN        NaN  False
5            5     liv_ri       constant -0.492528  0.342118  -1.439644   True
6            6     liv_ri    needs_level  1.122217  0.436828   2.569016   True
7            7     liv_ri  prob_help_adl -0.371731  0.195417  -1.902239   True
8            8     liv_ri  help_adl_miss -0.196654  0.179189  -1.097465   True
9            9     liv_ri      ln_income  0.000000       NaN        NaN  False
10          10  liv_chsld       constant -1.660407  0.269203  -6.167851   True
11          11  liv_chsld    needs_level  2.456677  

In [27]:
init_pars.table()

,group,label,value,stderr,tvalue,free
0,liv_rpa,constant,-0.93857,NaN,NaN,True
1,liv_rpa,needs_level,0.833992,NaN,NaN,True
2,liv_rpa,prob_help_adl,-0.21765,NaN,NaN,True
3,liv_rpa,help_adl_miss,-0.163541,NaN,NaN,True
4,liv_rpa,ln_income,0.0,NaN,NaN,True
5,liv_ri,constant,-0.492528,NaN,NaN,True
6,liv_ri,needs_level,1.122217,NaN,NaN,True
7,liv_ri,prob_help_adl,-0.371731,NaN,NaN,True
8,liv_ri,help_adl_miss,-0.196654,NaN,NaN,True
9,liv_ri,ln_income,0.0,NaN,NaN,True


In [28]:
model = logit(with_uh = True)
model.set_params(init_pars)
for s in sago.attr_labels:  
    model.set_shifters(s,shifters)
model.set_choices(cs)

#ctr = [0,0,0,0,0,1,1,1,1,-1]
ctr = [0,0,0,0,0,0,0,0,0,0]
#ctr = [0,0,0,1,1,1,1]
ctrs = dict(zip(sago.attr_labels,ctr))
model.set_attributes(attributes, sago.attr_labels, ctrs)
model.set_draws(n_d = 100)

In [29]:
model.estimate(scn_name ='uh_test',algorithm='BFGS')
#model.estimate(scn_name ='uh1',algorithm='BFGS')

iter =  0 , loglike =  -4.0834450119945105
current theta:
[-9.38700045e-01  8.33911877e-01 -2.17678324e-01 -1.63549530e-01
 -6.96756112e-04 -4.92474086e-01  1.12226056e+00 -3.71717242e-01
 -1.96655017e-01  2.29786581e-04 -1.66038246e+00  2.45670313e+00
 -5.80678893e-01 -5.47997702e-01 -8.37108896e-05 -4.06255169e-01
  8.58582905e-05 -5.11538586e-05 -1.90456010e-04  1.02584214e-01
  6.66510776e-03 -1.18587281e-04 -1.86922445e-04  1.84737296e-04
 -7.64214700e-05  3.37899559e-01 -3.80488172e-01  1.87814419e-01
 -4.61857846e-02  1.42019452e-02  3.23046751e-01 -2.34170468e-01
  2.41982020e-02 -6.46598688e-02 -2.03403297e-03  1.75822172e-01
 -3.98341750e-01  1.57624941e-01 -3.75885961e-02  7.29768968e-02
  2.49244277e-01 -4.01471390e-01  1.87421277e-01  7.31275372e-02
  9.82698600e-02 -2.29586927e-01  3.53922712e-05  6.42584568e-05
  4.93650569e-06 -7.20134262e-05 -2.30253305e+00 -2.30257662e+00
 -2.30253557e+00 -2.30242215e+00 -2.30241018e+00 -2.30244208e+00
 -2.30249558e+00 -2.30256148e+00

In [30]:
model.pars.table()

,group,label,value,stderr,tvalue,free
0,liv_rpa,constant,-1.043638,0.262721,-3.972416,True
1,liv_rpa,needs_level,0.896394,0.154917,5.7863,True
2,liv_rpa,prob_help_adl,-0.232528,0.129795,-1.791499,True
3,liv_rpa,help_adl_miss,-0.175531,0.116355,-1.508577,True
4,liv_rpa,ln_income,-0.036585,0.065173,-0.561359,True
5,liv_ri,constant,0.070588,0.814426,0.086672,True
6,liv_ri,needs_level,0.744575,0.666566,1.117032,True
7,liv_ri,prob_help_adl,-0.644788,0.333602,-1.932806,True
8,liv_ri,help_adl_miss,-0.454444,0.320857,-1.416347,True
9,liv_ri,ln_income,-0.080092,0.168369,-0.475693,True


In [15]:
model.pars.Sig[0][9]

0.0

In [89]:
df_sig = pd.DataFrame(index=range(10),columns=range(10))
for i in range(10):
    for j in range(10):
            df_sig.loc[i,j] = model.pars.Sig[i][j]

df_sig


,0,1,2,3,4,5,6,7,8,9
0,1.382098,1.48239,2.313143,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.48239,4.603705,6.068371,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.313143,6.068371,9.286139,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
np.linalg.cholesky(model.pars.Sig)

LinAlgError: Matrix is not positive definite

In [24]:
init_pars.table()

,group,label,value,stderr,tvalue,free
0,liv_rpa,constant,-28.240274,NaN,NaN,True
1,liv_rpa,needs_level,85.357672,NaN,NaN,True
2,liv_rpa,prob_help_adl,7.547916,NaN,NaN,True
3,liv_rpa,help_adl_miss,15.323811,NaN,NaN,True
4,liv_rpa,adl_miss,-65.814393,NaN,NaN,True
5,liv_rpa,ln_income,0.0,NaN,NaN,False
6,liv_ri,constant,66.654349,NaN,NaN,True
7,liv_ri,needs_level,-46.091705,NaN,NaN,True
8,liv_ri,prob_help_adl,-6.349881,NaN,NaN,True
9,liv_ri,help_adl_miss,-4.622242,NaN,NaN,True
